In [0]:
"""from google.colab import drive
drive.mount('/content/drive')"""
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab Notebooks/



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks


In [0]:
cd Experiments/exp_2/

/gdrive/My Drive/Colab Notebooks/Experiments/exp_2


In [0]:
!ls

 casia_ela	    exp_2.ipynb   test7.ipynb
 demo_exp_2.ipynb   sample_data  'Untitled document.gdoc'


In [0]:
import os
import io
import numpy as np
from keras.preprocessing import image
import pandas as pd
from PIL import Image, ImageChops, ImageEnhance
import os
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.models import Model, Sequential 
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau, EarlyStopping




Using TensorFlow backend.


In [0]:
def jpeg_compression(img: Image.Image, quality: int=90) -> Image.Image:
    if img.mode != 'RGB':
        img = img.convert('RGB')
    buffer = io.BytesIO()
    img.save(buffer, format='JPEG', quality=quality)
    im_data = buffer.getvalue()
    img = Image.open(io.BytesIO(im_data))
    buffer.close()
    return img


In [0]:
def convert_to_ela(img):
    im = img.convert('RGB')
    resaved_im = jpeg_compression(im)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [0]:
data_path = "casia_ela/"

In [0]:
#Defining Parameters
batch_size = 64
nb_epochs = 20
img_width = 256
img_height = 256


In [0]:
model = Sequential()

# input dimensions 

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'valid',activation ='relu', input_shape=(img_height, img_width, 3)))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)
 

model.add(MaxPooling2D(pool_size = (2, 2)))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)



model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'valid', activation ='relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(MaxPooling2D(pool_size=(2,2)))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Dropout(0.25))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape, "  <----------")



#def tf_spatial_pyramid_pooling(tf_input_feature_maps, tf_spatial_pyramid, dtype=tf.float32):
#    return tf.py_func(np_spatial_pyramid_pooling, [tf_input_feature_maps, tf_spatial_pyramid], dtype)
#average pooling !!

#from keras.layers import Input
#from keras.models import Model
#from keras_pyramid_pooling_module import PyramidPoolingModule

#input_ = Input((224, 224, 3))
#x = PyramidPoolingModule()(input_)
#model = Model(inputs=input_, outputs=x)








#model.add(GlobalAveragePooling2D())
#print("Input: ", model.input_shape)
#print("Output: ", model.output_shape, " <-----------")


model.add(Flatten())
print("Input: ", model.input_shape)
print("Output: ", model.output_shape, "final2")

model.add(Dense(256, activation = "relu"))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)
    

model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape, "final2")



Input:  (None, 256, 256, 3)
Output:  (None, 252, 252, 32)
Input:  (None, 256, 256, 3)
Output:  (None, 126, 126, 32)
Input:  (None, 256, 256, 3)
Output:  (None, 122, 122, 32)
Input:  (None, 256, 256, 3)
Output:  (None, 61, 61, 32)
Input:  (None, 256, 256, 3)
Output:  (None, 61, 61, 32)   <----------
Input:  (None, 256, 256, 3)
Output:  (None, 119072) final2
Input:  (None, 256, 256, 3)
Output:  (None, 256)
Input:  (None, 256, 256, 3)
Output:  (None, 2) final2


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 252, 252, 32)      2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 122, 32)      25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 61, 61, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 119072)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

In [0]:
optimizer = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)

In [0]:
model.compile(optimizer = optimizer , loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
#weights = "saved_weights/test7.hdf5"
#model.load_weights(weights)


In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [0]:
train_data = train_datagen.flow_from_directory(
    data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

Found 11469 images belonging to 2 classes.


In [0]:
validation_data = train_datagen.flow_from_directory(
    data_path, # same directory as training data
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 2866 images belonging to 2 classes.


In [0]:
history = model.fit_generator(
            train_data,
            steps_per_epoch = train_data.samples // batch_size,
            validation_data = validation_data, 
            validation_steps = validation_data.samples // batch_size,
            epochs = nb_epochs)

Epoch 1/20
 93/179 [==============>...............] - ETA: 28:39 - loss: 0.5838 - accuracy: 0.7245

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 80 could not be retrieved. It could be because a worker has died.
  UserWarning)


 96/179 [===============>..............] - ETA: 28:02 - loss: 0.5771 - accuracy: 0.7275

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 115 could not be retrieved. It could be because a worker has died.
  UserWarning)


165/179 [==========================>...] - ETA: 4:40 - loss: 0.5005 - accuracy: 0.7673

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 86 could not be retrieved. It could be because a worker has died.
  UserWarning)


179/179 [==============================] - 4226s 24s/step - loss: 0.4900 - accuracy: 0.7726 - val_loss: 1.4248 - val_accuracy: 0.6460
Epoch 2/20
179/179 [==============================] - 230s 1s/step - loss: 0.3398 - accuracy: 0.8545 - val_loss: 2.5811 - val_accuracy: 0.6324
Epoch 3/20
179/179 [==============================] - 219s 1s/step - loss: 0.2966 - accuracy: 0.8705 - val_loss: 0.9301 - val_accuracy: 0.6720
Epoch 4/20
179/179 [==============================] - 218s 1s/step - loss: 0.2711 - accuracy: 0.8852 - val_loss: 1.0837 - val_accuracy: 0.6834
Epoch 5/20
179/179 [==============================] - 217s 1s/step - loss: 0.2692 - accuracy: 0.8910 - val_loss: 0.6018 - val_accuracy: 0.6738
Epoch 6/20
179/179 [==============================] - 216s 1s/step - loss: 0.2470 - accuracy: 0.8960 - val_loss: 1.0574 - val_accuracy: 0.6613
Epoch 7/20
179/179 [==============================] - 216s 1s/step - loss: 0.2411 - accuracy: 0.8994 - val_loss: 0.8276 - val_accuracy: 0.6924
Epoch 8/

In [0]:
print("done")

done


In [0]:
w_test7 = "saved_weights/test7.hdf5"
model.save_weights(w_test7, overwrite = True)

O

In [0]:
print(history.history.keys())

In [0]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
#Confusion Matrix and Classification Report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
Y_pred = model.predict_generator(validation_data, validation_data.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_data.classes, y_pred))


print('Classification Report')
target_names = ['Aut', 'Tamp']
print(classification_report(validation_data.classes, y_pred, target_names=target_names))

In [0]:
x, y = validation_data.class_indices
print(x, y)

In [0]:
#path of any test image
predict_path = "predict_images/i1_cm1.jpg"



In [0]:
from IPython.display import Image as imgloader
display(imgloader(filename = predict_path, width = img_width, height=img_height))


In [0]:
test_img = image.load_img(predict_path, target_size = (img_width, img_height))


# if the image is an ela map already then comment the following 
test_img = convert_to_ela(test_img)



temp = test_img
test_image = image.img_to_array(test_img)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)
temp


In [0]:
train_data.class_indices

In [0]:
result

In [0]:
tamp = result[0][1]
if(tamp==1):
    print("Fake image, some tampering detected !")
else:
    print("The image looks authentic !")

In [0]:
!wget "https://storage.googleapis.com/defacto-db/inpainting/annotations.zip?GoogleAccessId=automailsystem%40defacto-231809.iam.gserviceaccount.com&Expires=1593674746&Signature=UjB8MeZs8ZBiRGT8TuoMI2e%2BH66ePjXmGq9fXCc6ghxQ8LqCWVYXyogJbFvVPDCCPj2mgn5m9JJJjyo3QN95B48wViu4au%2BxXbP34SWTHz6gb%2Fby%2FzyOWT9YriCCl%2BI%2BYDMwBYq34nH%2F5b4YJNxXseAcs7iRwOg0XcpX7%2BntAmW%2FXCOwV53EN0KBTwBI0JT71QoozssaI9SG%2B5QossMn6af7KrjwO95tIv6y5re7JeRDVtjgoNQRTEpbSdqKLuqIRLym2ZX%2F9WbfwjIbzj9ldTvBiAGLmhM5eN5ty6GsfsyfgM%2BBXdmkRh4HVS4DqDIb0nzT5eU2sc6JsPb4wEpfQw%3D%3D" -O random_data.tar

--2020-06-08 13:29:05--  https://storage.googleapis.com/defacto-db/inpainting/annotations.zip?GoogleAccessId=automailsystem%40defacto-231809.iam.gserviceaccount.com&Expires=1593674746&Signature=UjB8MeZs8ZBiRGT8TuoMI2e%2BH66ePjXmGq9fXCc6ghxQ8LqCWVYXyogJbFvVPDCCPj2mgn5m9JJJjyo3QN95B48wViu4au%2BxXbP34SWTHz6gb%2Fby%2FzyOWT9YriCCl%2BI%2BYDMwBYq34nH%2F5b4YJNxXseAcs7iRwOg0XcpX7%2BntAmW%2FXCOwV53EN0KBTwBI0JT71QoozssaI9SG%2B5QossMn6af7KrjwO95tIv6y5re7JeRDVtjgoNQRTEpbSdqKLuqIRLym2ZX%2F9WbfwjIbzj9ldTvBiAGLmhM5eN5ty6GsfsyfgM%2BBXdmkRh4HVS4DqDIb0nzT5eU2sc6JsPb4wEpfQw%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65695187 (63M) [application/zip]
Saving to: ‘random_data.tar’

random_data.tar     100%[===================>]  62.65M  66.3MB/s    in 0.9s    

2020-06-08 13:29:06 (66.3 MB/s) - ‘ran

In [0]:
!cd

In [0]:
!pwd

/gdrive/My Drive/Colab Notebooks
